Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

# LAB10

Use reinforcement learning to devise a tic-tac-toe player.

### Deadlines:

* Submission: [Dies Natalis Solis Invicti](https://en.wikipedia.org/wiki/Sol_Invictus)
* Reviews: [Befana](https://en.wikipedia.org/wiki/Befana)

Notes:

* Reviews will be assigned  on Monday, December 4
* You need to commit in order to be selected as a reviewer (ie. better to commit an empty work than not to commit)

In [74]:
from itertools import permutations
import numpy as np

In [75]:
from abc import ABC, abstractmethod


class Player(ABC):
    def __init__(self) -> None:
        """You can change this for your player if you need to handle state/have memory"""
        pass

    @abstractmethod
    def make_move(self, game: "TicTacToe") -> str:
        """
        game: the Quixo game. You can use it to override the current game with yours, but everything is evaluated by the main game
        return values: this method shall return a tuple of X,Y positions and a move among TOP, BOTTOM, LEFT and RIGHT
        """
        pass

In [76]:
class TicTacToe:
    def __init__(self, board=None):
        if board != None:
            self.board = board
        else:
            self.board = np.zeros(shape=(3, 3))

    def check_win(self):
        """Checks if someone won the game."""
        check_win_by_col = (abs(self.board.sum(axis=0)) == 3).any()
        check_win_by_row = (abs(self.board.sum(axis=1)) == 3).any()

        if check_win_by_col or check_win_by_row:
            return True

        sum_diag_princ = 0
        sum_diag_back = 0
        for i in range(3):
            sum_diag_back += self.board[i][i]
            sum_diag_princ += self.board[2 - i][i]

        if abs(sum_diag_back) == 3 or abs(sum_diag_princ) == 3:
            return True

        return False

    def make_move(self, player: int, move: tuple[int, int]):
        """Take a move in format x y and makes it."""
        x, y = move
        # check is a valid move
        if not (0 <= x <= 2 and 0 <= y <= 2):
            print("invalid move")
        elif self.board[x][y] != 0:
            print("invalid move")
        else:
            self.board[x][y] = player

    def possible_moves(self):
        """Return all the possible available moves to make."""
        moves = []
        for i in range(3):
            for j in range(3):
                if self.board[i][j] == 0:
                    moves.append((i, j))

        return moves

    def print(self):
        for i in range(3):
            print("----------")
            print(
                " | ".join(
                    map(
                        lambda e: " " if e == 0 else "x" if e == 1 else "o",
                        self.board[i].astype(int),
                    )
                )
            )
        print("----------")

    def game(self, player1: Player, player2: Player):
        # TODO at each move we could save the state of the board in a list.
        board_state = []
        someone_won = False
        self.board = np.zeros(shape=(3, 3))
        # We need to also consider ties!!
        # Possible moves = []
        while not someone_won:
            possible_moves = self.possible_moves()

            pass

In [79]:
game = TicTacToe()
game.print()
game.check_win()

----------
  |   |  
----------
  | o |  
----------
  |   |  
----------


False